
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/088_Temporal_Fusion_Transformer_TFT.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/88_Temporal_Fusion_Transformer_TFT.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# ⏳ TFT: Transformer do zadań specjalnych (Time Series)

Standardowy Transformer (GPT) traktuje wszystko jako tekst.
TFT jest zaprojektowany specjalnie dla liczb i czasu.

Rozwiązuje problem **Heterogenicznych Danych**:
1.  **Zmienne statyczne:** (ID sklepu, lokalizacja) -> Nie zmieniają się w czasie.
2.  **Zmienne dynamiczne znane:** (Dzień tygodnia, Święta) -> Znamy je na rok do przodu.
3.  **Zmienne dynamiczne nieznane:** (Sprzedaż) -> Znamy tylko przeszłość.

**Kluczowa innowacja: Gating (Bramkowanie).**
Większość sieci neuronowych to "czarne skrzynki". TFT używa mechanizmu **GLU (Gated Linear Unit)**, który działa jak kran. Może całkowicie odciąć dopływ informacji z danej kolumny, jeśli uzna ją za szum.

Zbudujemy od zera serce TFT: **Gated Residual Network (GRN)**.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HIDDEN_DIM = 64  # Rozmiar ukryty (dla każdego feature'a)
DROPOUT = 0.1

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Krok 1: GLU (Gated Linear Unit)

To prosty, ale genialny mechanizm.
$$ GLU(x) = \sigma(W_1 x + b_1) \odot (W_2 x + b_2) $$

*   Część prawa ($W_2 x$): Przetwarza dane (Informacja).
*   Część lewa ($\sigma(...)$): Sigmoid zwraca wartości 0-1 (Bramka).

Mnożymy Informację przez Bramkę. Jeśli Bramka = 0, informacja znika.

In [2]:
class GLU(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # Wersja PyTorchowa GLU oczekuje wejścia 2x większego, 
        # bo dzieli je na pół (jedna połowa to dane, druga to bramka).
        self.linear = nn.Linear(input_dim, input_dim * 2)

    def forward(self, x):
        # x: [Batch, Dim]
        val = self.linear(x)
        # F.glu dzieli tensor na pół i robi: A * sigmoid(B)
        return F.glu(val, dim=-1)

# Test
glu = GLU(HIDDEN_DIM)
dummy = torch.randn(5, HIDDEN_DIM)
out = glu(dummy)
print(f"Wejście: {dummy.shape}")
print(f"Wyjście: {out.shape} (Wymiar zachowany, ale przefiltrowany)")

Wejście: torch.Size([5, 64])
Wyjście: torch.Size([5, 64]) (Wymiar zachowany, ale przefiltrowany)


## Krok 2: GRN (Gated Residual Network)

To jest podstawowy klocek TFT (używany wszędzie).
Składa się z:
1.  **Skip Connection:** Oryginał dodawany na końcu (pamiętasz ResNet?).
2.  **LayerNorm:** Stabilizacja.
3.  **Dwie warstwy Linear + ELU:** Nieliniowe przetwarzanie.
4.  **GLU:** Bramkowanie na końcu.
5.  **Context (Optional):** GRN może przyjmować dodatkowy wektor kontekstu (np. "To jest Sklep nr 5"), który wpływa na przetwarzanie.

$$ GRN(x, c) = LayerNorm(x + GLU(Linear(ELU(Linear(x, c))))) $$

In [3]:
class GRN(nn.Module):
    def __init__(self, input_dim, hidden_dim, context_dim=None):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Warstwa 1
        # Jeśli mamy kontekst, doklejamy go (lub rzutujemy)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        if context_dim is not None:
            self.context_projection = nn.Linear(context_dim, hidden_dim, bias=False)
            
        # Warstwa 2
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Bramka i Normalizacja
        self.glu = GLU(hidden_dim)
        self.norm = nn.LayerNorm(hidden_dim)
        
        # Projekcja rezydualna (jeśli wejście ma inny wymiar niż wyjście)
        self.skip_projection = nn.Linear(input_dim, hidden_dim) if input_dim != hidden_dim else nn.Identity()

    def forward(self, x, context=None):
        # x: [Batch, Input_Dim]
        residual = self.skip_projection(x)
        
        # 1. Pierwsza warstwa + Kontekst
        x = self.fc1(x)
        if context is not None:
            # Dodajemy kontekst (np. wektor statyczny sklepu) do przetwarzania
            x = x + self.context_projection(context)
            
        x = F.elu(x) # Exponential Linear Unit (standard w TFT)
        
        # 2. Druga warstwa
        x = self.fc2(x)
        
        # 3. Bramkowanie (GLU) + Dropout
        x = F.dropout(x, p=DROPOUT, training=self.training)
        x = self.glu(x)
        
        # 4. Add & Norm
        return self.norm(x + residual)

# Test z Kontekstem
grn = GRN(input_dim=10, hidden_dim=64, context_dim=5)
x_in = torch.randn(32, 10) # 32 próbki, 10 cech
c_in = torch.randn(32, 5)  # Kontekst (np. ID sklepu)

out = grn(x_in, c_in)
print(f"GRN Output: {out.shape}")

GRN Output: torch.Size([32, 64])


## Krok 3: Variable Selection Network (VSN)

To jest unikalne dla TFT.
Zamiast wrzucać wszystkie cechy do jednego worka (jak w MLP), TFT przetwarza **każdą kolumnę osobno** przez własny GRN.
Na końcu sieć decyduje (waży), które kolumny są ważne dla danej próbki.

Dzięki temu TFT jest **Interpretowalny**. Powie Ci: *"Dla tej prognozy wzięłam pod uwagę 80% Sprzedaży Wczorajszej i 20% Pogody, a zignorowałam Dzień Tygodnia"*.

In [4]:
class VariableSelectionNetwork(nn.Module):
    def __init__(self, num_inputs, input_dim, hidden_dim, context_dim=None):
        super().__init__()
        self.num_inputs = num_inputs # Ile mamy kolumn (zmiennych)?
        
        # Dla każdej zmiennej tworzymy osobny GRN
        self.single_variable_grns = nn.ModuleList([
            GRN(input_dim, hidden_dim, context_dim) for _ in range(num_inputs)
        ])
        
        # GRN ważący (decyduje o wagach dla każdej zmiennej)
        # Wejście to spłaszczone wszystkie zmienne
        self.weighting_grn = GRN(num_inputs * input_dim, num_inputs, context_dim)
        
    def forward(self, x_list, context=None):
        # x_list: Lista tensorów (każdy to jedna zmienna np. [Batch, 1])
        # Musimy je najpierw zrzutować na ten sam wymiar (Embedding), tu pomijamy dla uproszczenia
        # Zakładamy, że x_list to tensor [Batch, Num_Inputs, Input_Dim]
        
        batch_size = x_list.shape[0]
        
        # 1. Przetwarzamy każdą zmienną przez jej GRN
        processed_vars = []
        for i in range(self.num_inputs):
            var_out = self.single_variable_grns[i](x_list[:, i, :], context)
            processed_vars.append(var_out)
            
        processed_vars = torch.stack(processed_vars, dim=1) # [Batch, Num, Hidden]
        
        # 2. Obliczamy wagi ważności (Weights)
        # Spłaszczamy wejście dla Weighting GRN
        flat_input = x_list.view(batch_size, -1)
        weights = self.weighting_grn(flat_input, context)
        weights = F.softmax(weights, dim=-1) # [Batch, Num_Inputs]
        
        # 3. Suma ważona
        # weights: [Batch, Num, 1]
        weights = weights.unsqueeze(-1)
        combined = torch.sum(processed_vars * weights, dim=1)
        
        return combined, weights

# Symulacja: Mamy 3 zmienne (np. Sprzedaż, Pogoda, Cena), każda ma wymiar 64 (po embeddingu)
vsn = VariableSelectionNetwork(num_inputs=3, input_dim=64, hidden_dim=64)

dummy_vars = torch.randn(32, 3, 64) # [Batch, Zmienne, Cechy]
out, weights = vsn(dummy_vars)

print(f"Wyjście VSN: {out.shape} -> Jeden wektor reprezentujący cały krok czasowy.")
print("--- WAGI WAŻNOŚCI (Feature Importance) dla pierwszego przykładu ---")
print(weights[0].squeeze().detach().numpy())

Wyjście VSN: torch.Size([32, 64]) -> Jeden wektor reprezentujący cały krok czasowy.
--- WAGI WAŻNOŚCI (Feature Importance) dla pierwszego przykładu ---
[0.08056269 0.12061661 0.79882073]


## 🧠 Podsumowanie: Dlaczego TFT jest SOTA?

TFT łączy zalety wszystkich światów:
1.  **RNN (LSTM):** Używa ich do lokalnego przetwarzania sekwencji (nie pokazaliśmy tego tutaj, ale są w pełnej architekturze).
2.  **Transformer (Attention):** Używa Multi-Head Attention do patrzenia na długoterminowe zależności (np. "sprzedaż rok temu").
3.  **Drzewa Decyzyjne (Selection):** Dzięki `VariableSelectionNetwork` potrafi odrzucać szum, co zwykle robią XGBoosty.

Dlatego TFT wygrywa konkursy forecastingowe (np. M5 Competition) i jest używany w Google Cloud Forecasting.